<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

## Exercise: Explicit Memory Spaces

Usage of `thrust::copy` for your reference:

```c++
thrust::copy(src_vector.begin(), src_vector.end(), dst_vector.begin());
```

Rewrite the code below to use `thrust::device_vector` and `thrust::host_vector`:

<details>
<summary>Original code in case you need to refer to it.</summary>

```c++
%%writefile Sources/heat-2D-explicit-memory-spaces.cu
#include "ach.h"

int main()
{
  int height = 4096;
  int width  = 4096;

  thrust::universal_vector<float> prev = ach::init(height, width);
  thrust::universal_vector<float> next(height * width);

  for (int write_step = 0; write_step < 3; write_step++) {
    std::printf("   write step %d\n", write_step);
    ach::store(write_step, height, width, prev);
    
    for (int compute_step = 0; compute_step < 3; compute_step++) {
      auto begin = std::chrono::high_resolution_clock::now();
      ach::simulate(height, width, prev, next);
      auto end = std::chrono::high_resolution_clock::now();
      auto seconds = std::chrono::duration<double>(end - begin).count();
      std::printf("computed step %d in %g s\n", compute_step, seconds);
      prev.swap(next);
    }
  }
}
```
    
</details>

In [ ]:
%%writefile Sources/heat-2D-explicit-memory-spaces.cu
#include "ach.h"

int main()
{
  int height = 4096;
  int width  = 4096;

  thrust::universal_vector<float> prev = ach::init(height, width);
  thrust::universal_vector<float> next(height * width);

  for (int write_step = 0; write_step < 3; write_step++) {
    std::printf("   write step %d\n", write_step);
    ach::store(write_step, height, width, prev);
    
    for (int compute_step = 0; compute_step < 3; compute_step++) {
      auto begin = std::chrono::high_resolution_clock::now();
      ach::simulate(height, width, prev, next);
      auto end = std::chrono::high_resolution_clock::now();
      auto seconds = std::chrono::duration<double>(end - begin).count();
      std::printf("computed step %d in %g s\n", compute_step, seconds);
      prev.swap(next);
    }
  }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/heat-2D-explicit-memory-spaces.cu # build executable
!/tmp/a.out # run executable

If you’re unsure how to proceed, consider expanding this section for guidance. Use the hint only after giving the problem a genuine attempt.

<details>
  <summary>Hints</summary>
  
  - `ach::simulate` should accept `thrust::device_vector` as input and output
  - `ach::store` should accept `thrust::host_vector` as input
  - You can use `thrust::copy` to copy data between `thrust::device_vector` and `thrust::host_vector`
</details>

Open this section only after you’ve made a serious attempt at solving the problem. Once you’ve completed your solution, compare it with the reference provided here to evaluate your approach and identify any potential improvements.

<details>
  <summary>Solution</summary>

  Key points:

  - 

  Solution:
  ```c++
  thrust::device_vector<float> d_prev = ach::init(height, width);
  thrust::device_vector<float> d_next(height * width);
  thrust::host_vector<float> h_prev(height * width);

  for (int write_step = 0; write_step < 3; write_step++) {
    std::printf("   write step %d\n", write_step);
    thrust::copy(d_prev.begin(), d_prev.end(), h_prev.begin());
    ach::store(write_step, height, width, h_prev);

    for (int compute_step = 0; compute_step < 3; compute_step++) {
      auto begin = std::chrono::high_resolution_clock::now();
      ach::simulate(height, width, d_prev, d_next);
      auto end = std::chrono::high_resolution_clock::now();
      auto seconds = std::chrono::duration<double>(end - begin).count();
      std::printf("computed step %d in %g s\n", compute_step, seconds);
      d_prev.swap(d_next);
    }
  }
  ```

  You can find full solution [here](Solutions/heat-2D-explicit-memory-spaces.cu).
</details>


---

Great job!  Understanding of the memory spaces concept concludes the first part of the course.
Proceed to [the summary](../01.07-Summary/01.07.01-Summary.ipynb).

<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">